# How to present CV with Neptune

## Introduction

When training models with cross-validation, you can use Neptune namespaces to organize, visualize and compare models.

By the end of this guide, you will learn how to organize your run to track cross-validation metadata, so that you can easily analyze the results.

[See this example in Neptune](https://app.neptune.ai/o/common/org/showroom/e/SHOW-27624)

<center>
    <a href="https://app.neptune.ai/o/common/org/showroom/e/SHOW-27624">
        <img src="https://files.gitbook.com/v0/b/gitbook-x-prod.appspot.com/o/spaces%2F-MT0sYKbymfLAAtTq4-t%2Fuploads%2FjP2eBYeGVoqg2rfOkrA4%2FScreenshot%202022-07-06%20at%206.40.16%20PM.png?alt=media&token=3fd7ebf3-0494-4843-8954-53fb21357e1" alt="Drawing" style="height: 600px;">
    </a>
</center>
<center><small>CV results presented in Neptune UI</small></center>



## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects) 
<br><br>
* This example assumes: 
    * Basic familiarity with Neptune ([create run and log metadata to it](https://docs.neptune.ai/you-should-know/what-can-you-log-and-display)),
    * Familiarity with cross-validation techniques in machine learning.




## Install Neptune and dependencies

In [ ]:
! pip install neptune-client torch==1.10.2 torchvision==0.11.3 scikit-learn==1.0.2

## Step 1: Initialize Neptune and create new run

Connect your script to Neptune application and create new run.

In [ ]:
import neptune.new as neptune

run = neptune.init(
    project="common/showroom",
    api_token="ANONYMOUS",
    tags=["Colab Notebook", "cross-validation"],
)

Click on the link above to open this run in Neptune.

For now it is empty but keep the tab with run open to see what happens next. 

**Few explanations**

In the above code You tell Neptune:

* **who you are**: your Neptune API token `api_token`
* **where you want to send your data**: your Neptune `project`.

At this point you have new run in Neptune. For now on you will use `run` to log metadata to it.

---

**Note**

Instead of logging data to the public project 'common/showroom' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your [Neptune API token](https://docs-beta.neptune.ai/administration/security-and-privacy/how-to-find-and-set-neptune-api-token)
2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your project to the `project` argument of the `init()` method.

For example:
```python
neptune.init(project="YOUR_WORKSPACE/YOUR_PROJECT", 
             api_token="YOUR_API_TOKEN")
```

## Step 2: Log config and hyperparameters

### Log Hyperparameters

In [ ]:
parameters = {
    "epochs": 3,
    "learning_rate": 1e-2,
    "batch_size": 10,
    "input_size": 32 * 32 * 3,
    "n_classes": 10,
    "k_folds": 5,
    "checkpoint_name": "checkpoint.pth",
    "seed": 42,
}

In [ ]:
run["parameters"] = parameters

### Log Config
Model and Dataset

In [ ]:
import torch.nn as nn


class BaseModel(nn.Module):
    def __init__(self, input_sz, hidden_dim, n_classes):
        super(BaseModel, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_sz, hidden_dim * 2),
            nn.ReLU(),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, n_classes),
        )

    def forward(self, input):
        x = input.view(-1, 32 * 32 * 3)
        return self.main(x)

In [ ]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
torch.manual_seed(parameters["seed"])
model = BaseModel(
    parameters["input_size"], parameters["input_size"], parameters["n_classes"]
).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=parameters["learning_rate"])

Log model, criterion and optimizer name

In [ ]:
run["parameters/model/name"] = type(model).__name__
run["parameters/model/criterion"] = type(criterion).__name__
run["parameters/model/optimizer"] = type(optimizer).__name__

In [ ]:
from torchvision import datasets, transforms

data_dir = "data/CIFAR10"
compressed_ds = "./data/CIFAR10/cifar-10-python.tar.gz"
data_tfms = {
    "train": transforms.Compose(
        [
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )
}

In [ ]:
trainset = datasets.CIFAR10(data_dir, transform=data_tfms["train"], download=True)

validset = datasets.CIFAR10(
    data_dir, train=False, transform=data_tfms["train"], download=True
)

dataset_size = len(trainset)

Log dataset details

In [ ]:
run["dataset/CIFAR-10"].track_files(data_dir)
run["dataset/transforms"] = data_tfms
run["dataset/size"] = dataset_size

## Step 3: Log losses and metrics per fold 
Training Loop

In [ ]:
from sklearn.model_selection import KFold

splits = KFold(n_splits=parameters["k_folds"], shuffle=True)
epoch_acc_list, epoch_loss_list = [], []

In [ ]:
from torch.utils.data import SubsetRandomSampler, DataLoader

for fold, (train_ids, _) in enumerate(splits.split(trainset)):
    train_sampler = SubsetRandomSampler(train_ids)
    train_loader = DataLoader(
        trainset, batch_size=parameters["batch_size"], sampler=train_sampler
    )
    for epoch in range(parameters["epochs"]):
        epoch_acc, epoch_loss = 0, 0.0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model.forward(x)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, y)
            acc = (torch.sum(preds == y.data)) / len(x)

            # Log batch loss and acc
            run[f"fold_{fold}/training/batch/loss"].log(loss)
            run[f"fold_{fold}/training/batch/acc"].log(acc)

            loss.backward()
            optimizer.step()

        epoch_acc += torch.sum(preds == y.data).item()
        epoch_loss += loss.item() * x.size(0)

    epoch_acc_list.append((epoch_acc / len(train_loader.sampler)) * 100)
    epoch_loss_list.append(epoch_loss / len(train_loader.sampler))

    # Log model checkpoint
    torch.save(model.state_dict(), f"./{parameters['checkpoint_name']}")
    run[f"fold_{fold}/checkpoint"].upload(parameters["checkpoint_name"])

In [ ]:
from statistics import mean

# log mean acc and loss
run["results/metrics/train/mean_acc"] = mean(epoch_acc_list)
run["results/metrics/train/mean_loss"] = mean(epoch_loss_list)

## Stop run

**Warning**

Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
run.stop()

## Explore the run in the Neptune UI

After running the code cell in **Step 1**, you will get a link on the cell output similar to https://app.neptune.ai/o/common/org/showroom/e/SHOW-27624 with: 
* **common/showroom** replaced by **your_workspace/your_project**,
* **SHOW-27624** replaced by your *Run ID*. 

**Click on the link to open the Run in Neptune UI.**

<img src="https://files.gitbook.com/v0/b/gitbook-x-prod.appspot.com/o/spaces%2F-MT0sYKbymfLAAtTq4-t%2Fuploads%2FZkx5SZk10r7NvXu9VqdW%2Fper-fold-metadata.gif?alt=media&token=49373e21-4b6f-4962-a03b-73652ee8fd15" alt="Drawing" style="width: 100%;"/>
<center><small>Analysing per-fold metadata</small></center>

## Cross-validation with Integrations
If you are using Neptune with XGBoost or LightGBM you can get the structure for cross-validation automatically, by using available integrations.
<div style="position: relative; padding-bottom: 62.5%; height: 0;"><iframe src="https://www.loom.com/embed/98dc6247c65f49b8baf7476cf996dbe4" frameborder="0" webkitallowfullscreen mozallowfullscreen allowfullscreen style="position: absolute; top: 0; left: 0; width: 100%; height: 100%;"></iframe></div>

## Conclusion

You learned how to organize your run to track cross-validation metadata with Neptune and how to present the result in the Neptune UI for further comparison and analysis. 

Visit our docs for more tutorials and guides on how to use Neptune: https://docs.neptune.ai
